In [15]:
import requests
import json
import datetime
import sentencepiece
#%pip install sentencepiece
#%pip install torch


In [16]:
#Get last week's news
last_week = datetime.datetime.now() - datetime.timedelta(days=7)
url = (
    'https://newsapi.org/v2/everything?'
    'q="Nomura Holdings"&'  # Use double quotes to ensure the search is more precise
    'from=' + last_week.strftime('%Y-%m-%d') + '&'  # Start date for the news
    'sortBy=popularity&'  # Sort results by popularity
    'apiKey=1fd881e0e51d4752a86d6bdb7590201b'  # Your API key
)

response = requests.get(url)
data = response.json()

#Print the news
print('Nomura news from the last week:')
for article in data['articles']:
    print(article['title'])
    print(article['url'])
    print()

Nomura news from the last week:
Private-credit salaries: See what 16 nonbank lenders, from Blackstone to Apollo, are paying at all levels
https://www.businessinsider.com/private-credit-salaries-compensation-analyst-associate-vp-principal-more-2025-1

Ex-Archegos CFO Patrick Halligan sentenced to 8 years after costly collapse roiled Wall Street
https://nypost.com/2025/01/27/business/ex-archegos-cfo-patrick-halligan-sentenced-to-8-years/

Rupee pain starts to show as India’s top airline pummeled
https://economictimes.indiatimes.com/markets/forex/rupee-pain-starts-to-show-as-indias-top-airline-pummeled/articleshow/117718845.cms

Shadow Banking Market Sees $134.5 Billion in Revenues by 2030 - Growth of FinTech Platforms and Peer-to-Peer Lending Drives Adoption of Shadow Banking Services
https://www.globenewswire.com/news-release/2025/01/31/3018677/28124/en/Shadow-Banking-Market-Sees-134-5-Billion-in-Revenues-by-2030-Growth-of-FinTech-Platforms-and-Peer-to-Peer-Lending-Drives-Adoption-of-Sh

In [17]:
# Summarize the news content
from transformers import AutoModelWithLMHead, AutoTokenizer

# Load the pre-trained tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-summarize-news")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-summarize-news")

# Iterate over each article in the data (for this example, we'll only look at the first 5 articles)
summaryList = []
for i, article in enumerate(data['articles']):
    if i >= 5:
        break
    # Encode the article content
    inputs = tokenizer.encode("summarize: " + article['content'], return_tensors="pt", max_length=512, truncation=True)
    
    # Generate the summary
    outputs = model.generate(inputs, max_length=300, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    
    # Decode the summary
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    summaryList.append(summary)
    # Print the summary
    print(summary)
    print()



c:\Users\zheng\anaconda3\Lib\site-packages\transformers\models\auto\modeling_auto.py:1468: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Private credit is booming, which has resulted in a hiring spree. Business Insider looked at visa data and job listings to see what's new in the country's economy. The report was based on a study by a private credit analyst. It also revealed that private credit is booming, which has resulted in a hiring spree.

former chief financial officer of Archegos Capital Management was sentenced on Monday to eight years in prison for his role in the firm's 2021 collapse, which cost Wall Street banks more than $1 billion. Archegos was a major player in the global financial crisis that led to the collapse of the global financial services giant in 2021. He was also the first person to be sentenced to eight years in prison for his role in the firm's collapse. Notably, his role in the firm's 2021 collapse cost banks over $1 billion.

after two years of calm, the rupees sudden slide is starting to strain Indian corporate earnings, sparking a rush among some companies to hedge their currency exposures. 

In [18]:
# Use sentiment analysis to determine the sentiment of the news
from transformers import pipeline

# Load the classification pipeline with the specified model
pipe = pipeline("text-classification", model="tabularisai/multilingual-sentiment-analysis")

# Iterate over each article in the summaryList (for this example, we'll only look at the first 5 articles)
sentimentList = []
for i, summary in enumerate(summaryList):
    if i >= 5:
        break
    # Classify the sentiment of the summary
    sentiment = pipe(summary)[0]['label']
    sentimentList.append(sentiment)
    # Print the sentiment
    print(sentiment)
    print()

Very Positive

Very Negative

Negative

Neutral

Neutral

